In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch

# 데이터 전처리 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)), # 이미지 크기 조정
    transforms.ToTensor(), # 이미지를 텐서로 변환
])

# ImageFolder를 사용해 전체 데이터셋 로드
data_dir = 'dataset/train/nobreak/nobreak'  # 데이터셋 경로
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# 데이터셋 크기 계산 및 분할
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size

# random_split으로 데이터셋 분할
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# 훈련 및 검증 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 데이터 로더를 이용한 데이터 접근 예시
for images, labels in train_loader:
    # 훈련 데이터셋을 사용한 학습 코드
    pass

for images, labels in test_loader:
    pass

In [3]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# 사전 훈련된 ResNet 모델 로드
model = models.resnet152(pretrained=True)

# 모델의 마지막 계층을 현재 데이터셋의 클래스 수에 맞게 조정
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 6) # 6개의 클래스, 수정 필요함 

# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

C:\Users\AJOU\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\AJOU\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to C:\Users\AJOU/.cache\torch\hub\checkpoints\resnet152-394f9c45.pth
100%|███████████████████████████████████████████████████████████████████████████████| 230M/230M [00:39<00:00, 6.08MB/s]


In [4]:
# 단순화된 훈련 루프 예제
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # 훈련 모드로 설정
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

Epoch 1, Loss: 1.2215594161640515
Epoch 2, Loss: 0.9732218402804751
Epoch 3, Loss: 0.8379425424518008
Epoch 4, Loss: 0.7121473875912753
Epoch 5, Loss: 0.712455277190064
Epoch 6, Loss: 0.6785627404848734
Epoch 7, Loss: 0.5618536111080286
Epoch 8, Loss: 0.46501322316400934
Epoch 9, Loss: 0.33285787295211444
Epoch 10, Loss: 0.32578700080965506


In [5]:
#model.eval()  # 모델을 평가 모드로 설정
correct = 0
total = 0

with torch.no_grad():  # 기울기 계산을 비활성화하여 메모리 사용량 줄이고 계산 속도 향상
    for images, labels in test_loader:  # test_loader는 테스트 데이터셋의 DataLoader
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct / total} %')

Accuracy of the network on the test images: 76.92307692307692 %


In [6]:
# 모델 저장
torch.save(model.state_dict(), 'model_best_resnet152.pth')